In [0]:
# def get_code_data(path):
#     for _, _, files in os.walk(path):
#         data = np.empty((0, 100))
#         for file in tqdm_notebook(files):
#             with open(path + file, 'rb') as f:
#                 array = pickle.load(f)
#                 for i in tqdm_notebook(range(len(array)), leave=False):
#                     data = np.concatenate((data, array[i].inputs), axis=0)
#     return data


In [0]:
# data = get_code_data(base_path + '/after_preprocess/')

In [0]:
# import ast

In [0]:
# ast.dump(ast.parse('if x >= 0:\n    x = 0'))

In [0]:
# dir(ast.parse('test_var = test_list[0]').body[0])

## Импорты

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
from tqdm import tqdm_notebook, trange
import pickle
import numpy as np
from IPython.display import clear_output

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [4]:
device

device(type='cuda', index=0)

# Предобработка

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pwd

/content


In [20]:
!head drive/My\ Drive/vkr/mapping.map

UnicodeDecodeError: ignored

In [0]:
base_path = '/content/drive/My Drive/vkr'
# code_batched = base_path + '/after_preprocess/after_preprocess.part0'

In [27]:
!ls {base_path}

after_preprocess  mapping.map


In [30]:
print('/content/drive/My\\ Drive/vkr')

/content/drive/My\ Drive/vkr


In [35]:
# list(os.walk('/content/drive'))

[('/content/drive', ['My Drive', '.Trash'], []),
 ('/content/drive/My Drive',
  ['docs',
   'Fallout1Saves',
   'M-Art Sync',
   'NotCommon',
   'whoision',
   'whoision (1)',
   'SaveTravel',
   'Зубр',
   'Colab Notebooks',
   'Play Books Notes',
   'hse_nlp_hw_shared',
   'hse_nlp_hw2',
   'hse-cv',
   'vkr'],
  ['Создание игр.gdoc',
   'Полезные ссылки.gdoc',
   'Fallout.gdoc',
   'Dropbox.gdoc',
   'Ссылочки-ссылки.gdoc',
   'Ссылки_Август.gdoc',
   'SavePlaces.gdoc',
   'Books that should be read.gdoc',
   'Истории о разных маленьких мальчиках и девочках.gdoc',
   'Mr. Diary.gdoc',
   'PlayStation 3.gdoc',
   'Amazon_Conversation.gdoc',
   'Citilink.gdoc',
   'Game Making Tree.gdraw',
   'Субкультуры.gdoc',
   'Программирование с Денисом и Андреем.gdoc',
   'Реализм Толстого.gdoc',
   '(Before editing) My Top and Movies.gdoc',
   'Разговор с правым берегом.gdoc',
   'My Games for Trade.gdoc',
   "Game Designer's Tips.gdoc",
   "Game Designer(?)'s Diary.gdoc",
   'Games should be 

In [0]:
def build_vocab():
    # load pre-computed vocab
    with open(base_path + '/mapping.map', 'rb') as f:
#         print(base_path + '/mapping.map')
        word_to_id = pickle.load(f)
    id_to_word = dict([(v, k) for (k, v) in word_to_id.items()])
    return word_to_id, id_to_word

word_to_id, id_to_word = build_vocab()


In [0]:

params = {
    'batch_size': 64,
    'emb_size': 150,
    'vocab_size': len(word_to_id),
    # 'seq_len': 100,
}


In [0]:
# def get_code_data(path):
#     for _, _, files in os.walk(path):
#         data = np.empty((0, 100))
#         for file in tqdm_notebook(files):
#             with open(path + file, 'rb') as f:
#                 array = pickle.load(f)
#                 for i in tqdm_notebook(range(len(array)), leave=False):
#                     data = np.concatenate((data, array[i].inputs), axis=0)
#     return data


In [0]:
# data = get_code_data(base_path + '/after_preprocess/')

In [0]:
# data.shape

In [0]:
# data.astype('int64')

In [0]:
# with open('./after_preprocess/all_inputs', 'wb') as f:
#     pickle.dump(data.astype('int64'), f)

In [0]:
data_path = base_path + '/after_preprocess/all_inputs'

In [0]:
class CodeDataset(Dataset):
    def __init__(self, path, params):
        with open(path, 'rb') as f:
            self.data = pickle.load(f)
#         print(type(self.data))
#         print(type(self.data[0]))
#         self.batch_size = params['batch_size']

    def __len__(self):
#         # return number of batches
#         return int(np.ceil(self.data.shape[0] // self.batch_size))

#         return 3
        return self.data.shape[0]
    
    def __getitem__(self, index):
#         return self.data[index * self.batch_size : (index+1) * self.batch_size]
        return self.data[index]


In [0]:
train_loader = DataLoader(
    CodeDataset(data_path, params),
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=10
)

In [0]:
# data[0:64].shape

In [0]:
# for i in train_loader:
#     print(type(i))
#     print(i.shape)
#     break

# Инициализация модели

In [0]:
# !pip3 freeze

In [0]:
class LstmBaseline(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.params = params
        self.embedding = nn.Embedding(
            self.params['vocab_size'],
            self.params['emb_size']
        )
        self.lstm = nn.LSTM(
            self.params['emb_size'],
            self.params['emb_size'],
#             2
            1
        )
        self.linear = nn.Linear(
            self.params['emb_size'],
            self.params['vocab_size']
        )
    
    def init_hidden(self, batch_size):
        pass

    def forward(self, inputs, hidden):
        embs = self.embedding(inputs)
#         print(hidden.shape)
#         print('embs.shape', embs.shape)
#         print(embs.grad)
        # need 100, 64, 150(?)
        # real 64, 100, 150
#         print('inputs.shape', inputs.shape)
        output, hidden = self.lstm.forward(embs, hidden)
#         print(output.grad)
        output = self.linear(output)
#         print(output.grad)
        return output, hidden


# Обучение

In [50]:
print(torch.from_numpy(np.arange(3)).grad)

None


In [0]:
def train_epoch(model, optimizer, lr):
    loss_log = []
    model.train()
    
    for batch_num, x in zip(trange(len(train_loader)), train_loader):
        optimizer.zero_grad()
#         print(1)
#         print('x', x.grad)
#         print('x', type(x))
#         print('x', x)
#         x = x.long()
#         x.requires_grad = True

        x = x.to(device)
        loss_value = 0.
        loss = nn.CrossEntropyLoss()
        # todo check
        hidden = None
        for i in range(x.shape[1] - 1):
            output, hidden = model.forward(x[:, i].unsqueeze(1), hidden)
            loss_value += loss(output.squeeze(1), x[:, i + 1])
        loss_value.backward()

#         torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
#         for p in model.parameters():
#             p.data.add_(-lr, p.grad.data)
        optimizer.step()

        
        loss_value = loss_value.item()
        loss_log.append(loss_value)# / x.shape[1])
    return loss_log

# def test(model, test_batches):
#     loss_log = []
#     model.eval()
#     for batch_num, x in zip(trange(len(train_loader)), val_loader):        
#         hidden = model.init_hidden(batch.shape[0])
#         loss = 0
#         output = model.forward(x, y)
#         loss = F.cross_entropy(output.float(), y.float())
#         loss = loss.item()
#         loss_log.append(loss / nums.shape[0])
#     return loss_log

def plot_history(train_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    plt.xlabel('train steps')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    
def train(model, opt, n_epochs):
    train_log = []
#     val_log = []
#     perp_log = []
    lr = 0.05
    lr_decay_base = 1 / 1.15
    m_flat_lr = 20.0
    for epoch in range(n_epochs):
        lr_decay = lr_decay_base ** max(epoch - m_flat_lr, 0)
        lr = lr * lr_decay
        train_loss = train_epoch(model, opt, lr)
        train_log.extend(train_loss)
#         perp_log.append(perp)
        if (epoch + 1) % 10 == 0:
            torch.save(
                {
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': opt.state_dict(),
                    'loss': train_log[-1]
                },
                './model_checkpoints/lstm_baseline_checkpoint_{}.pt'.format(epoch)
            )
        clear_output()
        print("Epoch:{}".format(epoch))
        plot_history(train_log)
#         plot_history(perp_log)
    torch.save(
        {
            'epoch': n_epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': train_log[-1]
        },
        './model_checkpoints/lstm_baseline_checkpoint_{}.pt'.format(n_epochs)
    )
#     np.save("/home/.../model_checkpoints..._logs.npy", np.array(train_log))
#     np.save("/home/.../model_checkpoints..._logs_perp.npy", np.array(perp_log))
    
# model = LstmBaseline(params)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# train(model, optimizer, 10)


In [0]:
model = LstmBaseline(params).to(device)

In [0]:
%%time

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train(model, optimizer, 10)

  4%|▍         | 436/10871 [05:25<2:10:47,  1.33it/s]